In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark= SparkSession.builder.config("spark.jars.packages","mysql:mysql-connector-java:8.0.28").getOrCreate()

In [3]:
emp = spark.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/hr_db").option("driver","com.mysql.cj.jdbc.Driver").option("user","root").option("password","Itversity2022").option("dbtable","employees").load()
emp.show()

+-----------+-----------+----------+--------+------------+----------+----------+--------+--------------+----------+-------------+
|employee_id| first_name| last_name|   email|phone_number| hire_date|    job_id|  salary|commission_pct|manager_id|department_id|
+-----------+-----------+----------+--------+------------+----------+----------+--------+--------------+----------+-------------+
|        100|     Steven|      King|   SKING|515.123.4567|1987-06-17|   AD_PRES|24000.00|          null|      null|           90|
|        101|      Neena|   Kochhar|NKOCHHAR|515.123.4568|1989-09-21|     AD_VP|17000.00|          null|       100|           90|
|        102|        Lex|   De Haan| LDEHAAN|515.123.4569|1993-01-13|     AD_VP|17000.00|          null|       100|           90|
|        103|  Alexander|    Hunold| AHUNOLD|590.423.4567|1990-01-03|   IT_PROG| 9000.00|          null|       102|           60|
|        104|      Bruce|     Ernst|  BERNST|590.423.4568|1991-05-21|   IT_PROG| 6000.00| 

In [4]:
emp.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- job_id: string (nullable = true)
 |-- salary: decimal(8,2) (nullable = true)
 |-- commission_pct: decimal(2,2) (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)



In [4]:
sch = StructType([StructField("department_id",IntegerType(),True),StructField("managerId",StringType(),True),StructField("locationId",IntegerType(),True)])

In [5]:
df = spark.readStream.format("socket").option("host","localhost").option("port",2222).load()

In [8]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
valueDF = df.select(from_json(col("value").cast("string"), sch).alias("value"))

In [10]:
valueDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- department_id: integer (nullable = true)
 |    |-- managerId: string (nullable = true)
 |    |-- locationId: integer (nullable = true)



In [7]:
tradeDF = valueDF.select("value.*")
    

In [12]:
tradeDF.printSchema()

root
 |-- department_id: integer (nullable = true)
 |-- managerId: string (nullable = true)
 |-- locationId: integer (nullable = true)



In [8]:
j1= emp.join(tradeDF,emp.department_id==tradeDF.department_id,"inner")

In [ ]:
j1.writeStream\
      .format("console").outputMode("update")\
      .option("checkpointLocation", "chk134").trigger(processingTime='1 minute')\
      .start().awaitTermination()